In [ ]:
import numpy as np
import chipwhisperer as cw
import os

In [ ]:
%run "functions.ipynb"

In [ ]:
data_dir = "d:/data_store"

## ChipWhisperer Configuration (Shared)

In [ ]:
# Set hardware settings
SCOPETYPE = 'OPENADC'
PLATFORM = 'CW308_SAM4S'
CRYPTO_TARGET='TINYAES128C' # 'TINYAES128C' or 'MBEDTLS'
SS_VER='SS_VER_2_1'

In [ ]:
# Connect to ChipWhisperer
%run "../../../Setup_Scripts/Setup_Generic.ipynb"

### Firmware Programming

In [ ]:
# program firmware onto target
cw.program_target(scope, prog, "./firmwares/fw-120mhz-clkout.hex")

In [ ]:
#Baud is lower so it works from internal oscillator which isn't as precise
target.baud = 38400

In [ ]:
scope.io.hs2 = None

In [ ]:
scope.io.target_pwr = False
time.sleep(0.25)
scope.io.target_pwr = True

## Mixer Measurement - Clock from Device Cleaned Up

In [ ]:
def tcapture(prefix, syncstr, fmcu, msps):
    N = 10000
    group1, group2 = capture_ttest(N, picoscope=False, splot=splot)
    nptsave(prefix + "{:d}mhz_{:s}_{:d}msps_ttest_10k".format(fmcu, syncstr, msps), group1, group2)

In [ ]:
import time
scope.clock.clkgen_src = "extclk"

In [ ]:
scope.clock.pll.set_outfreqs(60E6, 60E6, 2, True)

Check that you've got a clkgen_freq around 60MHz, adc_freq around 120MHz, and freq_ctr is measuring the clock on HS1 as around 60MHz. Things won't be perfect due to the SAM4S2A internal oscillator.

If you don't see the right frequencies, check jumpers on the isolator board.

In [ ]:
scope.clock

In [ ]:
scope.io.hs2 = "clkgen"

In [ ]:
scope.gain.mode = "low"
scope.gain.gain = 45

In [ ]:
scope.adc.samples = 10000

In [ ]:
splot = cw.StreamPlot()
splot.plot()

In [ ]:
tcapture("mixer_bothclks_defaultfilter_", "sync", 120, 120)

In [ ]:
scope.adc.samples = 5000

In [ ]:
waves, textins, textouts, keys = capture_cpa(5000, splot=splot)

In [ ]:
save_ets(waves, textins, textouts, keys, "d:/data_store/mixer_default_120mhz_sync_120msps_cpa.ets", overwrite=False)

### Adjusting the PLL Loop Bandwidth

The CW-Husky uses a CDCI6214, see [datasheet here](https://www.ti.com/lit/ds/symlink/cdci6214.pdf?ts=1711988669125) we write register `1F` to set the PLL loop parameters. 

* ZeroCap value: 0x10 (480pF)
* Resistor value: 0x09 (8.5kOhm)
* PoleCap value: 0x14 (19.5pF)

This is roughly based on calculations from the TI tools for a lower loop bandwidth than the default.

In [ ]:
pll_loop_filt = 0x10 << 9 | 0x9 << 5 | 0x14 << 0
scope.clock.pll.write_reg(0x1f, pll_loop_filt)

In [ ]:
scope.adc.samples = 10000

In [ ]:
tcapture("mixer_bothclks_slowerfilter_", "sync", 120, 120)

In [ ]:
scope.adc.samples = 5000

In [ ]:
waves, textins, textouts, keys = capture_cpa(5000, splot=splot)

In [ ]:
save_ets(waves, textins, textouts, keys, "d:/data_store/mixer_slowerfilter_120mhz_sync_120msps_cpa.ets", overwrite=False)

## V-Div ADC Mixer

* Remove jumper from CWHS2 to MIX-LOW
* Connect SMA to VDIV


In [ ]:
scope.io.hs2 = None

In [ ]:
scope.gain.mode = "high"
scope.gain.gain = 30

In [ ]:
tcapture("vdiv_", "sync", 120, 120)

In [ ]:
scope.adc.samples = 5000

In [ ]:
waves, textins, textouts, keys = capture_cpa(40000, splot=splot)

In [ ]:
save_ets(waves, textins, textouts, keys, "d:/data_store/vdiv_120mhz_sync_120msps_cpa.ets", overwrite=False)

## Optical Isolator Testing

The following uses the optical isolator for T-Test & CPA result saving. For these results the 50 MBaud isolators are used. The CLKOUT is maximum of 60 MHz, so the faster isolators aren't needed.

You can avoid mounting U1 and TR1 on the boards (along with all the support parts for those devices).

Switch the isolator for the optical board. The HS2 optical connection is not needed, but the other optical connections are needed.

On the target isolator board:
 * Jumper HS1 to the 50MB isolator (the clock runs at 60MHz and seems to work OK still, the faster isolator isn't needed).
 * Connect a 3xAA battery holder

On the CW isolator board:
 * Jumper HS2 to MIXER-RF (this will connect the cleaned up clock from ChipWhisperer to the mixer)
 * Jumper 50MB to MIXER-LO
 * Jumper HS1 to MIXER-LO (this connects the optical isolator to HS1 input on ChipWhisperer)
 
Check the extclock `freq_ctr` is stable at around 60MHz & the `adc_freq` is again close to 120MHz:

In [ ]:
scope.clock

Turn on the clock to the mixer on the isolator board. If you run this from above you'll be using the tuned PLL parameters still.


In [ ]:
scope.io.hs2 = "clkgen"

In [ ]:
scope.gain.mode = "low"
scope.gain.gain = 50

In [ ]:
scope.adc.samples = 10000

In [ ]:
tcapture("opticalisolator_mixer_bothclks_", "sync", 120, 120)

In [ ]:
scope.adc.samples = 5000

In [ ]:
waves, textins, textouts, keys = capture_cpa(5000, splot=splot)

In [ ]:
save_ets(waves, textins, textouts, keys, "d:/data_store/opticalisolator_mixer_120mhz_sync_120msps_cpa.ets", overwrite=False)